## Model Analysis Notebook

In this notebook, we train, test, and evaluate the performance of an LSTM model in wind speed prediction and compare results to the persistence method, which is a common benchmark for wind speed prediction algorithms.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import sklearn
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

import tensorflow as tf
import keras
from keras.layers import Input, LSTM, Dense, Dropout, Flatten

In [ ]:
# Define how many time steps will be used in observation and prediction
n_past = 24 # The last day of data
n_future = 24 # The next day of data
n_features = 3

In [ ]:
# Define a function to split the series using a sliding window
def split_series(series, n_past=n_past, n_future=n_future, offset=0):
    X, y = list(), list()
    for i in range(int(len(series)/n_past)-1):
        X.append(series[i*n_past : i*n_past + n_past, :])
        y.append(series[offset + i*n_past + n_past : offset + i*n_past + n_past + n_future, :])
    return np.array(X), np.array(y)

In [ ]:
# Process and split the data for a site given its filename
def prep_data(filename, cy=2015):
    # Import the data for a single point
    data = pd.read_csv("Data/NOW-23 Great Lakes [2000-2020] 60min/" + filename, index_col=0)

    # Restrict the data to the last 5 years, giving us 4 years of training and 1 year of testing data
    data = data.iloc[int(len(data)*(cy-2000)/20):]

    # Split the data into training and testing samples
    cutoff = int(len(data)*0.8)
    test_data = data[cutoff:]
    data = data[:cutoff]
    
    # Designate which columns are used for training
    columns = [6, 7, 8]
    
    # Normalize the testing and training data
    test_data.iloc[:, columns], test_norms = normalize(test_data.iloc[:, columns], axis=0, norm='max', return_norm=True)
    data.iloc[:, columns], train_norms = normalize(data.iloc[:, columns], axis=0, norm='max', return_norm=True)

    # Split the data into series for training
    X_train, y_train = split_series(np.array(data.iloc[:, columns]), n_future=1, offset=24-1)
    X_test, y_test = split_series(np.array(test_data.iloc[:, columns]), n_future=1, offset=24-1)

    # Adjust the expected output to contain only the wind speed
    y_train, y_test = y_train[:, :, 2], y_test[:, :, 2]
    
    return X_train, y_train, X_test, y_test, train_norms, test_norms

In [ ]:
# Define the model architecture
def define_model():
    # Original model used for testing
    '''
    model = keras.models.Sequential()
    model.add(Input(shape=(n_past, n_features)))
    model.add(LSTM(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(Dense(1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)))
    model.compile(optimizer='adam', loss='mae')
    '''


    # Lighter model used for additional training
    model = keras.models.Sequential()
    model.add(Input(shape=(n_past, n_features)))
    model.add(LSTM(16, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(Dense(1, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)))
    model.compile(optimizer='adam', loss='mae')

    return model

In [ ]:
model = define_model()
model.summary()

In [ ]:
# Train one model with varying features
df = pd.DataFrame()
df['Epochs'] = list()
df['MAE'] = list()
for epoch in range(10, 310, 10):
    print(f"{epoch}")

    X_train, y_train, X_test, y_test, train_norms, test_norms = prep_data('7871.csv')
    model = define_model()
    model.fit(X_train,y_train,epochs=epoch,validation_data=(X_test,y_test),batch_size=128)
        
    predictions = model.predict(X_test)
    mae = mean_absolute_error(y_test[:, 0] * test_norms[2], predictions * test_norms[2])
        
    df.loc[len(df)+1] = [int(epoch), mae]

In [ ]:
plt.xlabel('Epochs Trained')
plt.ylabel('MAE')
plt.plot(df['Epochs'], df['MAE'])

In [ ]:
# Train models for every selected site
i = 1
for filename in os.listdir("Data/NOW-23 Great Lakes [2000-2020] 60min"):
    print(f"Point number {i} of 100")
    i += 1

    model = define_model()
    
    X_train, y_train, X_test, y_test, train_norms, test_norms = prep_data(filename, cy=2015)
    
    model.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test),batch_size=128)
    model.save("Data/Models/" + filename[:-4] + ".keras")

In [ ]:
mae, sites = list(), list()

for filename in os.listdir("Data/Large Models"):

    X_train, y_train, X_test, y_test, train_norms, test_norms = prep_data(filename[:-6] + '.csv', cy=2015)
    model = keras.saving.load_model("Data/Models/" + filename)
    model.compile(optimizer='adam', loss='mae')
    
    predictions = model.predict(X_test)
    mae.append(mean_absolute_error(y_test[:, 0] * test_norms[2], predictions * test_norms[2]))
    sites.append(filename[:-6])
    print(mae[-1])
df = pd.DataFrame()
df['MAE'] = pd.Series(mae)
df['SiteID'] = pd.Series(sites)

In [ ]:
import seaborn as sns
plt.style.use('seaborn-v0_8-ticks')
plt.boxplot(df['MAE'], vert=False)
plt.title("Mean Absolute Error")
plt.show()

df['MAE'].hist(bins=20)
plt.title("Mean Absolute Error")
plt.show()

sns.kdeplot(df['MAE'], color="blue", fill=True)
plt.title("Mean Absolute Error")
plt.show()

In [ ]:
print(f"Mean: {np.average(df['MAE'])}")
print(f"Median: {np.median(df['MAE'])}")
print(f"Standard Deviation: {np.std(df['MAE'])}")
print(f"n: {len(df['MAE'])}")

In [ ]:
# Finally, we repeat this analysis with a persistence model that uses the wind speed from 24h before as a prediction, demonstrating the superiority of the LSTM model

mae, sites = list(), list()

for filename in os.listdir("Data/Models"):

    X_train, y_train, X_test, y_test, train_norms, test_norms = prep_data(filename[:-6] + '.csv')

    predictions = [x[-1] for x in X_test[:, :, -1]]
    mae.append(mean_absolute_error(y_test[:, 0] * test_norms[2], np.array(predictions) * test_norms[2]))
    sites.append(filename[:-6])

df1 = pd.DataFrame()
df1['MAE'] = pd.Series(mae)
df1['SiteID'] = pd.Series(sites)

In [ ]:
df1['MAE'].hist(bins=20)
plt.show()

sns.kdeplot(df1['MAE'], color="blue", fill=True)

In [ ]:
total = 0
for i in df1['MAE']:
    total += i
peristence_avg_mae = total/len(df1['MAE'])
print(f"Average MAE of the persistence model: {peristence_avg_mae}")
print(f"The persistence model has a higher MAE by {(peristence_avg_mae/np.average(df['MAE']) - 1) * 100}%")

Unsurpisingly, the average MAE of the persistence model is around 30% higher than the average LSTM model MAE over all sites